# VARS-TOOL Quick Start Tutorial

## Install

This is a simple tutorial show casing VARS-TOOL functionalities that are being implement in Python.Let's install the latest VARS-TOOL from GitHub:

In [1]:
!git clone https://github.com/vars-tool/vars-tool
!pip install vars-tool/.

Cloning into 'vars-tool'...


Processing c:\users\corde\giws\vars-tool\tutorial\vars-tool
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for varstool: filename=varstool-2.1-py2.py3-none-any.whl size=63326 sha256=cae7069ca48ea637a0445e2cbe42daa963c7207cbfa2bbee033a233c0da4e14d
  Stored in directory: c:\users\corde\appdata\local\pip\cache\wheels\ea\22\e6\94dfc4ddef885e03d97899daed3e63a6c2c3a1295f1b37f687
Successfully built varstool
  Attempting uninstall: varstool
    Found existing installation: varstool 2.1
    Uninstalling varstool-2.1:
      Successfully uninstalled varstool-2.1


## Example using the Ishigami and wavy6D models

Let's import VARS for the analysis and the Model class for creating a wrapper around the desired model so that it can be inputted into VARS

In [2]:
# importing VARS and Model from varstool package
from varstool import VARS, Model

# these are being imported in order for us to display the data nicely on the notebook
import numpy as np
import pandas as pd

Here is a quick function definition of the Ishigami model:

In [3]:
def ishigami(x, a=7, b=0.05):
    '''Ishigami test function'''
    # check whether the input x is a dataframe
    
    if not isinstance(x, (pd.core.frame.DataFrame, pd.core.series.Series, np.ndarray, list)):
        raise TypeError('`x` must be of type pandas.DataFrame, numpy.ndarray, pd.Series, or list')
    
    if len(x) > 3:
        raise ValueError('`x` must have only three arguments at a time')
    
    return np.sin(x[0]) + a*(np.sin(x[1])**2) + b*(x[2]**4)*np.sin(x[0])

Here is a quick function definition of the wavy6D model:

In [4]:
def wavy6D(x):
    '''wavy6D test function'''
    
# The features include:             
# (1) The 6 input factors are non-interacting                  
# (2) The function is "wavy" along the first 5 dimensions but with different 
#     frequencies and amplitude at different directions                                     
# (3) The last dimension (X6) is fully isensitive (dummy variable)                                         
# (4) It possess large-scale features such multi-modality (X1) 
#     to small-scale features such as roughness (X3)          
# (5) The directional variograms of X1 and X2 cross each other
    
    # check whether the input x is a dataframe
    
    if not isinstance(x, (pd.core.frame.DataFrame, pd.core.series.Series, np.ndarray, list)):
        raise TypeError('`x` must be of type pandas.DataFrame, numpy.ndarray, pd.Series, or list')
    
    if len(x) > 6:
        raise ValueError('`x` must have only five arguments at a time')
    
    y1 = -1*np.sin(2*np.pi*x[0]/2) - 0.3*np.sin(2*np.pi*x[0]/0.6);
    y2 = 0.76*(-1*np.sin(2*np.pi*(x[1]-0.2)/2)) - 0.315;
    y3 = 0.12*(-1*np.sin(2*np.pi*(x[2]-0.2)/1.9)) + 0.02*(-1*np.sin(2*np.pi*x[2]/0.021)) - 0.96;
    y4 = 0.12*(-1*np.sin(2*np.pi*(x[3]-0.2)/1.9))-0.97;
    y5 = 0.05*(-1*np.sin(2*np.pi*(x[4]-0.2)/2))-1.02;
    y6 = -1.08;
    
    return y1 + y2 + y3 + y4 + y5 + y6

As mentioned previously The `Model` class is a wrapper for custom functions(models) in the online version of VARS. However, the first paramter of every function must accept an array of parameters

Here we will create two wrappers for the testing models ishigami and wavy6D:

In [5]:
ishigami_model = Model(ishigami)

In [6]:
wavy6D_model = Model(wavy6D)

When creating an experiment you will need to assign a variable to an instance of VARS as shown below.

The paramters of the VARS class are described as:

**paramaters**: the name of each paramter along with their upper and lower bounds

**num_stars**: the total number of star samples that are desired for STAR-VARS analysis

**delta_h**: the sampling resolution of the VARS analysis

**ivars_scales**: the scales that are to be used when doing IVARS, e.g, 0.1 and 0.3 correspond (0-0.1) and (0-0.3) <br /> note: can not have a scale larger then 0.5

**star_centres**: This is only used if a sampler is not chosen and you are wanting to generate your own star centres(randomized numbers)

**sampler**: the sampling strategy: `rnd`, `lhs`, `plhs`, `sobol_seq`, or `halton_seq` for generation of star centers.

**seed**: the seed number for randomization of the sampling strategy specified by `sampler`, only needed if a sampler was chosen

**model**: the wrapper of your model you made when using the `Model` class

**bootstrap_flag**: this is a True/False value that specifies if bootstrapping will be used in the VARS analysis

**bootstrap_size**: the number of sampling iterations with replacement

**bootstrap_ci**: the bootstrap-based confidence interval on results

**grouping_flag**: this is a True/False value that specifies if paramater grouping will be used in the VARS analysis

**num_grps**: the number of groups you want to split your paramaters into, if left blank the optimal number of groups will be calculated by VARS

Create `experiment_1` and initialize the values needed to run a VARS analysis:

Run the next block if you want to run the notebook using the ishigami model, do not run it if you are using wavy6D

In [7]:
experiment_1 = VARS(parameters = {'x1':(0, 1), 'x2':(0, 1), 'x3': (0, 1)},
                    num_stars=10,
                    delta_h = 0.1,
                    ivars_scales = (0.1, 0.3, 0.5),
                    sampler = 'rnd',
                    seed = 123456789,
                    model = ishigami_model,
                    bootstrap_flag = True,
                    bootstrap_size = 100,
                    bootstrap_ci=0.9,
                    grouping_flag=True,
                    num_grps=2
                )

Run the next block if you want to run the notebook using the wavy6D model do not run it if you are using ishigami

In [8]:
experiment_1 = VARS(parameters = {'x1':(0, 1), 'x2':(0, 1), 'x3': (0, 1), \
                                  'x4':(0, 1), 'x5':(0, 1), 'x6': (0, 1)},
                    num_stars=10,
                    delta_h = 0.1,
                    ivars_scales = (0.1, 0.3, 0.5),
                    sampler = 'rnd',
                    seed = 123456789,
                    model = wavy6D_model,
                    bootstrap_flag = True,
                    bootstrap_size = 100,
                    bootstrap_ci=0.9,
                    grouping_flag=True,
                    num_grps=2
                )

A report displaying the current status of the VARS analysis can be found by typing in the variable name of the instance you created, here this is `experiment_1`

In [9]:
experiment_1

Star Centres: Loaded
Star Points: Not Loaded
Parameters: 6 paremeters set
Delta h: 0.1
Model: wavy6D
Seed Number: 123456789
Bootstrap: On
Bootstrap Size: 100
Bootstrap CI: 0.9
Grouping: On
Number of Groups: 2
VARS Analysis: Not Done

To run the VARS analysis simply do the following:

In [10]:
experiment_1.run_online()

ValueError: shape mismatch: value array of shape (3,)  could not be broadcast to indexing result of shape (3,6,6)

Now if we take a look at the status report again we can see that the `Vars Analysis` is now done.

In [ ]:
experiment_1

Now we can access all the results using 'dot' notation on our created instance:

**Directional variogram results**

There are 10 rows as the number of stars was 10, and each at a resolution incremented by 0.1 which was specificed to be the sampling resolution

In [ ]:
experiment_1.variogram_value.unstack(0)

**Directional covariogram results**

In [ ]:
experiment_1.covariogram_value.unstack(0)

**Directional expected covariogram results**

In [ ]:
experiment_1.e_covariogram_value.unstack(0)

**VARS-TO: Sobol total-order effects calculated through VARS**

In [ ]:
experiment_1.sobol_value

**VARS-ABE: Morris mean absolute elementary effects across scales**

In [ ]:
experiment_1.morris_value[0].unstack(0)

**VARS-ACE: Morris mean actual elementary effects across scales**

In [ ]:
experiment_1.morris_value[1].unstack(0)

**Integrated variogram**

In [ ]:
experiment_1.ivars

**The factor(parameter) rankings based on their influence**. 

The influence is based on how large or small a result is. The lower the ranking the more influential(larger) a factor is.

In [ ]:
experiment_1.ivars_factor_ranking

In [ ]:
experiment_1.sobol_factor_ranking

**Bootstrapping results based on the confidence interval limits**

low is the lower limit of the results after `bootstrap_size` sampling iterations

upp is the upper limit of the results after `bootstrap_size` sampling iterations

In [ ]:
experiment_1.variogram_low

In [ ]:
experiment_1.variogram_upp

In [ ]:
experiment_1.sobol_low

In [ ]:
experiment_1.sobol_upp

In [ ]:
experiment_1.ivars_low

In [ ]:
experiment_1.ivars_upp

**Reliability estimates of factor rankings based on VARS-TO**

reliability estimates give the ratio of how many times the rankings were the same for `bootstrap_size` sampling iterations

In [ ]:
experiment_1.rel_sobol_factor_ranking

In [ ]:
experiment_1.rel_ivars_factor_ranking

**Reliability estimates of factor rankings based on IVARS 50**

In [ ]:
experiment_1.ivars50_grp

In [ ]:
experiment_1.sobol_grp

**Reliability estimates of rankings based on grouping**

In [ ]:
experiment_1.reli_sobol_grp

In [ ]:
experiment_1.reli_ivars50_grp